In [1]:
%matplotlib inline
import xarray as xr
import os
import numpy as np
import matplotlib.pyplot as plt
import collections
import warnings 

This is an initial notebook for deriving new VIC 5 parameters in the RASM domain. 

It is a work in progress. 

The accompanying excel sheet `deriving_new_parameters_v2.xlsx` and Word Doc `procedure_for_derivation.docx` provide additional citations and details on methods. 

In [2]:
soil_data_vars = collections.OrderedDict()
soil_data_vars['silt'] = 'silt_sl*'
soil_data_vars['sand'] = 'sand_sl*'
soil_data_vars['clay'] = 'clay_sl*'
soil_data_vars['bulk_density'] = 'bulk_density_sl*'

# soil data dict with nlayer = 7 (base resolution of data)
soil_data = {}
soil_data_dir = '/u/home/gergel/data/parameters/soil_data/rasm_grid_netcdfs'
for soil_var, soil_wildcard in soil_data_vars.items(): 
    soil_data[soil_var] = xr.open_mfdataset(os.path.join(soil_data_dir, soil_wildcard),
                                            concat_dim='nlayer', 
                                            data_vars='all', 
                                            coords='all')

__If using a different domain than the 50km domain, change domain file in the following cell__ 

In [4]:
domain = xr.open_dataset(os.path.join('/u/home/gergel/data/parameters', 
                                      'domain.lnd.wr50a_ar9v4.100920.nc'))

__Set resolution for filenames__

In [4]:
res = '50km'

# set file extensions 
if res == "50km":
    file_ext = 'wr50a_ar9v4'
elif res == "25km":
    file_ext = 'wr25b_ar9v4'
elif res == "10km":
    raise ValueError("we do not have a domain file at %s so this option has not been implemented" %res)
print("calculating parameters at %s" %res)

calculating parameters at 50km


load CLM PFTs to use for vegetation parameters

In [5]:
# load regridded PFTs 
pfts_data_dir = '/u/home/gergel/data/parameters/pfts/regridded_pfts'
pfts_filename = 'mksrf_landuse_rc2000_c110913_wr50a_ar9v4.nc'
veg_data = xr.open_dataset(os.path.join(pfts_data_dir, pfts_filename))

define soil classification and lookup functions: `is_soil_class`, `is_param_value`, `classify_soil_texture`, and `soil_class_values` along with user-defined errors `SoilClassificationError` and `LookupTableError` for catching bad behavior in the above functions 

In [22]:
def is_soil_class(soil_class):
    '''
    Takes input soil classification and checks to make sure that it is greater than 0 
    (e.g. classified)
    '''
    assert soil_class > 0
        
def is_param_value(parameter):
    '''
    Takes input parameter and checks to be sure that it has been assigned a value 
    (should not be equal to 0 if it has been assigned)
    '''
    assert parameter > 0
    
class SoilClassificationError(Exception):
    pass

class LookupTableError(Exception):
    pass

def classify_soil_texture(sand, clay, silt):
    '''
    this function takes in percent sand and percent clay and classifies it in
    accordance with the ARS Soil Texture classes, listed here in VIC 5 Read the Docs: 
    https://vic.readthedocs.io/en/master/Documentation/soiltext/. 
    '''
    
    soil_class = 0
    
    # sand
    if silt + (1.5 * clay) < 15:
        soil_class = 1
    # loamy sand
    elif silt + (1.5 * clay) >= 15 and silt + (2 * clay) < 30:
        soil_class = 2
    # sandy loam
    elif ((clay >= 7 and clay < 20 and sand > 52 and silt + (2 * clay) >= 30) or 
          (clay < 7 and silt < 50 and silt + (2 * clay) >= 30)):
        soil_class = 3
    # loam
    elif clay >= 7 and clay < 27 and silt >= 28 and silt < 50 and sand <= 52:
        soil_class = 6
    # silt loam
    elif (silt >= 50 and clay >= 12 and clay < 27) or (silt >= 50 and silt < 80 and clay < 12):
        soil_class = 4
    # silt 
    elif silt >= 80 and clay < 12:
        soil_class = 5
    # sandy clay loam
    elif clay >= 20 and clay < 35 and silt < 28 and sand > 45:
        soil_class = 7
    # clay loam
    elif clay >= 27 and clay < 40 and sand > 20 and sand <= 45: 
        soil_class = 9
    # silty clay loam
    elif clay >= 27 and clay < 40 and sand <= 20:
        soil_class = 8
    # sandy clay 
    elif clay >= 35 and sand > 45: 
        soil_class = 10
    # silty clay
    elif clay >= 40 and silt >= 40: 
        soil_class = 11
    elif clay >= 40 and sand <= 45 and silt < 40:
        soil_class = 12
    elif np.isnan(clay):
        soil_class = 12
    
    return(soil_class)

def soil_class_values(soil_class, return_var):
    '''
    takes in a soil class (of type int) and uses lookup tables based on soil class for a number of 
    values that are either directly or indirectly used as VIC parameters. Decides which value to 
    return based on `return_var`, which is a string. 
    
    Valid values for `return_var` include: ksat, b, Wpwp_FRACT, Wcr_FRACT, resid_moist, quartz
    
    This is to get around the current issue with 
    returning multiple output arrays with xr.apply_ufunc. 
    
    Look-up values come from: 
    https://vic.readthedocs.io/en/master/Documentation/soiltext/ 
    Carsel and Parrish 1988, Table 3
    https://ral.ucar.edu/sites/default/files/public/product-tool/noah-multiparameterization-
    land-surface-model-noah-mp-lsm/soil_characteristics.html
    '''
    
    ksat = 0
    b = 0
    porosity = 0
    Wpwp_FRACT = 0
    Wcr_FRACT = 0
    resid_moist = 0
    quartz = 0
    
    # sand
    if soil_class == 1:
        ksat = 38.41
        b = 2.79
        porosity = 0.43
        Wpwp_FRACT = 0.03
        Wcr_FRACT = 0.7 * 0.08
        resid_moist = 0.045
        quartz = 0.92
    # loamy sand
    elif soil_class == 2: 
        ksat = 10.87
        b = 4.26
        porosity = 0.42
        Wpwp_FRACT = 0.06
        Wcr_FRACT = 0.7 * 0.15
        resid_moist = 0.057
        quartz = 0.82
    # sandy loam
    elif soil_class == 3:
        ksat = 5.24
        b = 4.74
        porosity = 0.4
        Wpwp_FRACT = 0.09
        Wcr_FRACT = 0.7 * 0.21
        resid_moist = 0.065
        quartz = 0.60
    # silty loam
    elif soil_class == 4:
        ksat = 3.96
        b = 5.33
        porosity = 0.46
        Wpwp_FRACT = 0.12
        Wcr_FRACT = 0.7 * 0.32
        resid_moist = 0.067
        quartz = 0.25
    # silty
    elif soil_class == 5:
        ksat = 8.59
        # NOTE: Table 3 doesn't have the class silty, using b for silty clay loam
        b = 8.72
        porosity = 0.52
        Wpwp_FRACT = 0.08
        Wcr_FRACT = 0.7 * 0.28
        resid_moist = 0.034
        quartz = 0.10
    # loam
    elif soil_class == 6:
        ksat = 1.97
        b = 5.25
        porosity = 0.43
        Wpwp_FRACT = 0.14
        Wcr_FRACT = 0.7 * 0.29
        resid_moist = 0.078
        quartz = 0.40
    # sandy clay loam
    elif soil_class == 7:
        ksat = 2.4
        b = 6.77
        porosity = 0.39
        Wpwp_FRACT = 0.17
        Wcr_FRACT = 0.7 * 0.27
        resid_moist = 0.100
        quartz = 0.60
    # silty clay loam
    elif soil_class == 8:
        ksat = 4.57
        b = 8.72
        porosity = 0.48
        Wpwp_FRACT = 0.21
        Wcr_FRACT = 0.7 * 0.36
        resid_moist = 0.089
        quartz = 0.10
    # clay loam
    elif soil_class == 9:
        ksat = 1.77
        b = 8.17
        porosity = 0.46
        Wpwp_FRACT = 0.21
        Wcr_FRACT = 0.7 * 0.34
        resid_moist = 0.095
        quartz = 0.35
    # sandy clay
    elif soil_class == 10:
        ksat = 1.19
        b = 10.73
        porosity = 0.41
        Wpwp_FRACT = 0.23
        Wcr_FRACT = 0.7 * 0.31
        resid_moist = 0.100
        quartz = 0.52
    # silty clay
    elif soil_class == 11:
        ksat = 2.95
        b = 10.39
        porosity = 0.49
        Wpwp_FRACT = 0.25
        Wcr_FRACT = 0.7 * 0.37
        resid_moist = 0.070
        quartz = 0.10
    # clay
    elif soil_class == 12:
        ksat = 3.18
        b = 11.55
        porosity = 0.47
        Wpwp_FRACT = 0.27
        Wcr_FRACT = 0.7 * 0.36
        resid_moist = 0.068
        quartz = 0.25
    elif np.isnan(soil_class):
        ksat = np.nan
        b = np.nan
        porosity = np.nan
        Wpwp_FRACT = np.nan
        resid_moist = np.nan
        quartz = np.nan
    else:
        ksat = 1.97
        b = 5.25
        porosity = 0.43
        Wpwp_FRACT = 0.14
        Wcr_FRACT = 0.7 * 0.29
        resid_moist = 0.078
        quartz = 0.40
        
    if ~np.isnan(soil_class):
        # correct for units of VIC 5 parameters
        ksat = ksat * 240 
        Wpwp_FRACT = Wpwp_FRACT / porosity 
        Wcr_FRACT = Wcr_FRACT / porosity
        
    
    if return_var == "ksat":
        return(ksat)
    elif return_var == "b":
        return(b)
    elif return_var == "Wpwp_FRACT":
        return(Wpwp_FRACT)
    elif return_var == "Wcr_FRACT":
        return(Wcr_FRACT)
    elif return_var == "resid_moist":
        return(resid_moist)
    elif return_var == "quartz": 
        return(quartz)

calculate soil types based on percent clay, percent sand and bulk density


In [14]:
# classify_soil_texture(sand, clay, silt)
soil_type_array = xr.apply_ufunc(classify_soil_texture, 
                                 soil_data['sand']['sand'].where(domain.mask == 1), 
                                 soil_data['clay']['clay'].where(domain.mask == 1),  
                                 soil_data['silt']['silt'].where(domain.mask == 1),
                                 dask='allowed',
                                 vectorize=True)

In [23]:
ksat = xr.apply_ufunc(soil_class_values, 
                      soil_type_array,
                      'ksat',
                      dask='allowed',
                      vectorize=True)
quartz = xr.apply_ufunc(soil_class_values, 
                        soil_type_array,
                        'quartz',
                        dask='allowed',
                        vectorize=True)
Wcr_FRACT = xr.apply_ufunc(soil_class_values, 
                           soil_type_array,
                           'Wcr_FRACT',
                           dask='allowed',
                           vectorize=True)
Wpwp_FRACT = xr.apply_ufunc(soil_class_values, 
                            soil_type_array,
                           'Wpwp_FRACT',
                            dask='allowed',
                            vectorize=True)
b = xr.apply_ufunc(soil_class_values, 
                   soil_type_array,
                   'b',
                   dask='allowed',
                   vectorize=True)
resid_moist = xr.apply_ufunc(soil_class_values, 
                             soil_type_array,
                             'resid_moist',
                             dask='allowed',
                             vectorize=True)

load regridded GTOPO 30 data, data var for elevation is called `Band1`, in (m)

In [33]:
gtopo_filename = 'sdat_10003_1_20180525_151136146_wr50a_ar9v4.nc'
gtopo = xr.open_dataset(os.path.join('/u/home/gergel/data/parameters/gtopo30', gtopo_filename))
elev = gtopo['Band1']

load regridded WORLDCLIM climate data for annual t and p 

In [31]:
clim_direc = '/u/home/gergel/data/parameters/world_clim_data/50km'
prec = xr.open_mfdataset(os.path.join(clim_direc, 'prec*'),
                                      concat_dim='time', 
                                      data_vars=['prec'], 
                                      coords='all')

# aggregate to annual, need average annual precip
annual_precip = prec['prec'].sum('time')

temp = xr.open_mfdataset(os.path.join(clim_direc, 'tavg*'),
                                      concat_dim='time', 
                                      data_vars='all', 
                                      coords='all')
tavg = temp['tavg'].mean('time')